In [2]:
from fuzzywuzzy import process
import re
from collections import deque
import itertools

class namecontext:
    def __init__(self, size):
        self.unique = set()
        self.queue = deque(maxlen=size)
        
    def __contains__(self, item):
        return item in self.unique

    def addcontext(self, name):
        if (name in self) or (name is None):
            return
        self.queue.append(name)
        self.unique = {x for x in self.queue}

def registerStreets(streets):
    streets["11th Street"] = "Eleventh Street"
    streets["2nd Avenue/Street"] = "Second Avenue/Street"
    streets["12th Street"] = "Twelfth Street"
    streets["8th Street"] = "Eight Street"
    streets["6th Street"] = "Sixth Street"
    streets["5th Street/Street"] = "fifth Street/Street"
    streets["3rd Avenue/Street"] = "Third Avenue/Street"
    streets["4th Avenue/Street"] = "Fourth Avenue/Street"
    streets["1st Street"] = "First Street"
    streets["10th Street"] = "Tenth Street"
    streets["7 Mile Road"] = "Seven Mile Road"
    streets["3 Mile Road"] = "Three Mile Drive"
    
    
class resultValue:
    def __init__(self, valor):
        self.value = valor
    
    
class streetfinder:
    def __init__(self, precision =60, size =10):
        self.regex = re.compile('^([0-9]+)[ -](.+)$')
        with  open('./data/Detroitstreet.txt', 'r') as f:
            allstreets =[t[:-1].strip() for t in f.readlines()]
            self.streets = {x: x for x in allstreets}
        registerStreets(self.streets)
        print(self.streets["7 Mile Road"])
        for key, value in self.streets.items():
            stripname = value.replace(' Avenue/Street','').replace('Avenue/Place','').replace('Avenue/Road','').replace(' Street','').replace(' Avenue','').replace(' Road','').replace(' Court','').replace(' Drive','').replace(' Place','')
            if (not stripname == key) and len(stripname)>2:
                self.streets[key] = stripname
        print(self.streets["7 Mile Road"])
        self.precision = precision
        self.cached = {}
        self.context = namecontext(size)
            
            
    def bestmatch(self, result):
        if len(result) ==0:
            return True, None
        if len(result) ==1:
            return True, result[0][2]
        for candidat in result:
            curr = candidat[2]
            if curr in self.context:
                return False, curr
        return False, candidat[2]
            
    def extractFirsts(self, query):
        best_list = process.extractBests(query, self.streets, score_cutoff= self.precision)
        if len(best_list) == 0:
            return best_list
        first_score = best_list[0][1]
        return list(itertools.takewhile(lambda x: x[1] == first_score, best_list))
    
    def getfromchache(self, name):
        res = self.cached.get(name)
        if res is not None:
            self.context.addcontext(res.value)
            return res
        return None
            
    def match(self, name):
        res = self.getfromchache(name)
        if res is not None:
            return res.value
        name = name.replace(' BLVD',' Boulevard').replace(' RD','Road').replace(' DR','Drive')
        res = self.getfromchache(name)
        if res is not None:
            self.cached[name] = resultValue(res.value)
            return res.value               
        allres = self.extractFirsts(name)
        unic, res = self.bestmatch(allres)
        if unic:
            self.cached[name] = resultValue(res)
        self.context.addcontext(res)
        return res
    
    def parse(self, streetaddress):
        m = self.regex.match(streetaddress)
        if m is not None:
            return (int(m.group(1)), self.match(m.group(2)))
        return (None, self.match(streetaddress))
    

sf = streetfinder( precision =90)

Seven Mile Road
Seven Mile


In [2]:
print(sf.parse('5519 THREE MILE DR'))
print(sf.parse('7575 CHRYSLER'))
print(sf.parse('1295 BALMORAL DR'))
print(sf.parse('6421 CYMBAL'))
print(sf.parse('12800 ST MARYS'))

16800 
process.extractBests('ST MARYS', sf.streets, limit=20)
process.extractBests('SCHAEFER HWY', sf.streets, limit=20)

(5519, '3 Mile Road')
(7575, 'Chrysler Service Drive')
(1295, 'Balmoral Avenue/Road')
(6421, 'Cymbal Street')
(12800, None)


[('Schaefer Highway', 86, 'Schaefer Highway'),
 ('Fern', 74, 'Fern Avenue'),
 ('Fern', 74, 'Fern Place'),
 ('Defer', 72, 'Defer Place'),
 ('Chase', 72, 'Chase Street'),
 ('Chase', 72, 'Chase Road'),
 ('Shaw', 68, 'Shaw Street'),
 ('Hale', 68, 'Hale Street'),
 ('Harper', 60, 'Harper Avenue'),
 ('Keifer', 60, 'Keifer Street'),
 ('Harper', 60, 'Harper Court'),
 ('Schley', 60, 'Schley Street'),
 ('Lee', 60, 'Lee Place'),
 ('Hafeli', 60, 'Hafeli Place'),
 ('Chapel', 60, 'Chapel Avenue'),
 ('Scheffer/Street', 59, 'Scheffer Place/Street'),
 ('Chartier', 57, 'Chartier Avenue'),
 ('Emery', 57, 'Emery Avenue'),
 ('Schweizer', 57, 'Schweizer Place'),
 ('Chandler', 57, 'Chandler Avenue')]

In [2]:
sf.streets['8 Mile Road']

'8 Mile'

In [3]:
import sqlite3
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
#query="select Id, Address, StreetName, StreetNumber from LocationReferenced where StreetName is null or StreetName='D Street' order by GeoHash;"
#query="select Id, Address, StreetName from LocationReferenced where StreetName='Green Avenue';"
query="select Id, Address, StreetName from DismantledLocationReferenced where StreetName is null;"
queryupdate = "update DismantledLocationReferenced set StreetName=:name, StreetNumber=:number where Id=:rid;"

count=1
rows = cur.execute(query).fetchall()[:]
for row in rows:
    rid, adress, snam = row
    if count ==1:
        print(rid, adress, snam)
        print(len(rows))
    number, name =  sf.parse(adress)
    if name is not None or number is not None:
        cur.execute(queryupdate, {'name':name,'number':number,'rid':rid})
        con.commit()
    if count%1000==1:
        print(rid)
        print('....')
    count+=1
con.close()

1 4331  BARHAM None
7133
1
....
1001
....
2001
....
3001
....
4001
....
5001
....
6001
....
7001
....


In [77]:
import sqlite3
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()

queryupdate = "update LocationReferenced set StreetName=null where Address in {};"

allfile = "("
first = True
with  open('./data/clean.txt', 'r') as f:
    for line in f.readlines():
        if not first:
            allfile +=', '
        allfile += "'"+ line[:-1].replace("'","''")+ "'"
        first=False

allfile+=" )"    
allfile

con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
query = queryupdate.format(allfile)
print(query)
cur.execute(query)
print(cur.rowcount)
con.commit()
con.close()

update LocationReferenced set StreetName=null where Address in ('19968 Five Points St', '8253 Desota St', 'Saint Mary''S St.', '2249 S. Kane', '5000 Conner Street 48213,', '52 Mount Vernon St', '18092  Greely St.', '6355 Miltary St', '1400 Mount Elliott Street 48207,', '5000 Conner Street 48213,', '3548 West Outwr Drive 48221', '4280 Seebaltd St Mi 48204', '19820 Praire St', '16509 Saint Marys Street', '2222 Mount Elliott Street 48207,', 'Drive Chrysler Freeway & Interstate 75 & Interstate 94', '11301-11325 Gietzen St', '11241 Kenmoor St', '11240-11248 Kenmoor St', '11248 Kenmoor St', '11240-11248 Kenmoor St', '11234 Kenmoor St', '638 W Euclifd', '18646 Saint Aubin Street 48234,', 'Fischer Street Wayne County, 48213, United States of America', '7752 Aubrun St.', 'Rochelle St', '439 Conner Street 48215,', '9741 Manisitique St.', '6509 Saint Marys Street', '11240 Kenmoor St', '11109 Kenmoor St', 'John C Lodge Freeway & Interstate 94 48202,', '4401-4459 Saint Antoine Street', '19901-20387